In [6]:
import os
import cv2,collections
import sys
import math
import random
import numpy as np
import plotly.express as px
from queue import Queue
import matplotlib.pyplot as plt
sys.setrecursionlimit(1000000)

In [7]:
video_name="2015_05_09_10_02_39FrontColor_IN"
directory="../images/"+video_name
if not os.path.exists(directory):
    os.makedirs(directory)
directoryf="../images/"+video_name+"/frames"
if not os.path.exists(directoryf):
    os.makedirs(directoryf)
directorym="../images/"+video_name+"/motion_vec"
if not os.path.exists(directorym):
    os.makedirs(directorym)
directoryj="../images/"+video_name+"/joined"
if not os.path.exists(directoryj):
    os.makedirs(directoryj)
directoryb="../images/"+video_name+"/joined_bl"
if not os.path.exists(directoryb):
    os.makedirs(directoryb)

In [8]:
vidObj = cv2.VideoCapture("../videos/IN/{}.avi".format(video_name))
count = 0
success = 1
sec = 0
while success:
    vidObj.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
    success, image = vidObj.read()
    if success == 1:
        cv2.imwrite(directoryf+"/frame%d.jpg" % count, image)
    count += 1
    sec += 0.1

In [10]:
count=177

In [11]:
feature_params = dict(maxCorners = 300, qualityLevel = 0.2, minDistance = 2, blockSize = 7)
lk_params = dict(winSize = (15,15), maxLevel = 2, criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
color = (0, 255, 0)
first_frame = cv2.imread(directoryf+"/frame0.jpg")
# first_frame = cv2.imread("images/trial1/frame60.jpg")

prev_gray = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)
prev = cv2.goodFeaturesToTrack(prev_gray, mask = None, **feature_params)
mask = np.zeros_like(first_frame)

T = 7.5          # threshold on the amount of movement recognized
K = 100            # the amount of seeds chosen for bfs

# takes position(i,j), visited matrix, motion vector matrix and threshold on vector magnitude
def bfs(i, j, vis, mag, t, region):
#     print(i,j)
    util = [-1, 0, 1]
    if mag[i][j] < t or vis[i][j] == True:
        return 0
    q = collections.deque()
    area = 0
    vis[i][j] = True
    q.append([i,j])
    while len(q) > 0:
        curX, curY = q.popleft()
        region[curX][curY] = 255
        area += 1
        for a in range(3):
            for b in range(3):
                x = curX + util[a]
                y = curY + util[b]
                if x >= 0 and y >= 0 and x < mag.shape[0] and y < mag.shape[1] and vis[x][y] != True:
                    vis[x][y] = True  
                    if mag[x][y] >= t:
                        q.append([x, y])
    return area
    
# plot = plt.figure(figsize=(100,100))

for index in range(1,count+1, 2):
    frame = cv2.imread(directoryf+"/frame"+ str(index)+".jpg")
#     frame = cv2.imread("images/trial1/frame"+ str(i)+".jpg")
    
    # plot.add_subplot(10, 2, 2*index-1)
    # plt.imshow(prev_gray, cmap='gray')
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    flow = gray.copy()
    prev = cv2.goodFeaturesToTrack(prev_gray, mask = None, **feature_params)

    # motion estimation
    flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 1, 5, 2, 5, 1.1, None)
    mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])

    # considering only vertical motion
    vertical = mag.copy()
    for i in range(mag.shape[0]):
        for j in range(mag.shape[1]):
            vertical[i][j] = mag[i][j] * abs(np.sin(ang[i][j]))

    # calculating varying T
    ind = 0
    temp_array = [0]*(vertical.shape[0]*vertical.shape[1])
    for i in range(vertical.shape[0]):
        for j in range(vertical.shape[1]):
            temp_array[ind] = vertical[i][j]
            ind += 1
    temp_array = np.array(temp_array)
    temp_array = np.sort(temp_array)[::-1]
    # top 5%
    percent = (2*mag.shape[0]*mag.shape[1])/100
    percent = int(percent)
    print(temp_array[percent-1])
    # ax = sns.heatmap(mag)
    # plt.show()
    T = temp_array[percent-1]
    if T < 2:
        T = 100
    # Calculating feature value M
    dy, dx = np.gradient(gray)
    Ixx = dx**2
    Iyy = dy**2
    Ixy = dx*dy
    R_score = np.array(gray.copy(),dtype=np.float64)
    for i in range(gray.shape[0]):
        for j in range(gray.shape[1]):
            sum_Ix = 0
            sum_Iy = 0
            sum_Ixy = 0
            for k in range(-2,3):
                for l in range(-2,3):
                    if i+k >= 0 and j+l >= 0 and i+k < gray.shape[0] and j+l < gray.shape[1]:
                        sum_Ix += Ixx[i+k][j+l]
                        sum_Iy += Iyy[i+k][j+l]
                        sum_Ixy += Ixy[i+k][j+l]
                    else:
                        sum_Ix += 0
                        sum_Iy += 0
                        sum_Ixy += 0
            det = sum_Ix*sum_Iy - sum_Ixy*sum_Ixy
            trace = sum_Ixy + sum_Ixy
            r = det - 0.05*(trace**2)
            R_score[i][j] = r 
    min_val = np.min(R_score)
    R_score += (-1)*min_val
    max_val = np.max(R_score)
    R_score = (R_score/max_val)*255
    R_score = R_score.astype(int)
    
#     print(np.max(ang), np.min(ang), "angles")
    # ax = sns.heatmap(R_score)
    # plt.show()
    listOfFeatures = []
    listTemp = []
    for x in range(frame.shape[0]):
        for y in range(frame.shape[1]):
            listTemp.append([R_score[x][y], x, y])
    listTemp.sort()
    listTemp.reverse()
#     print(listTemp[0],listTemp[-1])
    for x in range(len(listTemp)):
        listOfFeatures.append(listTemp[x])
    
    # break
    region = np.zeros((frame.shape[0], frame.shape[1]), dtype=int)
    vis = [[False for x in range(frame.shape[1])] for y in range(frame.shape[0])]
    for point in listOfFeatures:
        area = bfs(point[1], point[2], vis, vertical, T, region)
#         print(np.max(region), np.min(region))
        # do something with the area if we want to
    # plot.add_subplot(10, 2, 2*index)
    # plt.imshow(region, cmap='gray')
#     plt.show()
    
#     break
    prev_gray = gray.copy()
#     prev = good_new.reshape(-1, 1, 2)
    mask = np.zeros_like(frame)
    plt.imsave(directorym+'/frame'+ str(index)+'.jpg',region)

43.638012
48.081776
24.23752
17.760513
19.784065
18.184414
10.43749


In [ ]:
colors = []
for i in range(3000):
    colors.append(np.array([random.randint(0,255),random.randint(0,255),random.randint(0,255)]))

def color_it(img1,c):
    img=np.zeros((img1.shape[0],img1.shape[1]))
    img3=np.zeros((img1.shape[0],img1.shape[1],3),dtype=np.uint8)
    for i in range(img1.shape[0]):
        for j in range(img1.shape[1]):
            if img1[i][j] in c:
                img[i][j]=255
                for k in range(3):
                    img3[i][j][k]=colors[int(img1[i][j])][k]
    # plt.imshow(img3)
    # plt.show()
    return img,img3

def dfs22(i,j,img,re,vis,c):
    global ctr
    if(i<0 or i>=img.shape[0] or j<0 or j>=img.shape[1] or vis[i][j]==1 or img[i][j]==0):
        return 0
    else:
        vis[i][j]=1
        re[i][j]=c
        vall=1
        if img[i][j]!=0:
            vall+=dfs22(i+1,j,img,re,vis,c)
            vall+=dfs22(i-1,j,img,re,vis,c)
            vall+=dfs22(i,j+1,img,re,vis,c)
            vall+=dfs22(i,j-1,img,re,vis,c)
            vall+=dfs22(i+1,j+1,img,re,vis,c)
            vall+=dfs22(i-1,j-1,img,re,vis,c)
            vall+=dfs22(i-1,j+1,img,re,vis,c)
            vall+=dfs22(i+1,j-1,img,re,vis,c)
        return vall

def count_obj2(img,co=255):
    cc = 1
    dd = []
    re = np.zeros((img.shape[0],img.shape[1]))
    vi = np.zeros((img.shape[0],img.shape[1]))
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            if vi[i][j]==0 and img[i][j]==co:
                kkk=[]
                kkk.append(dfs22(i,j,img,re,vi,cc))
                kkk.append(cc-1)
                dd.append(kkk)
                cc+=1
    return cc-1,dd,re


In [ ]:
def join_disconnected(array_img_path,thr=50,thr1=3000,thr2=5000,ret=False):
    img_arr=[]
    # bbbb=[]
    for i in array_img_path:
        img = cv2.imread(i)
        j=i.replace("motion_vec","joined")
        jj=i.replace("motion_vec","joined_bl")
        img_gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        img_th=cv2.threshold(img_gray,50,255,cv2.THRESH_BINARY)[1]
        
        aa,bb,re=count_obj2(img_th,255)
        bbb=[x[0] for x in bb]
        aaa=[]
        aab=[]
        for i in bb:
            if i[0]>thr:
                aaa.append(i[0])
                aab.append(i[1]+1)
        imgc,img3c=color_it(re,aab)
        
        imgc[imgc>0]=255
        
        img_d=cv2.dilate(imgc,cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(7,7)),iterations=5)
        aa,bb,re=count_obj2(img_d,255)
        bbb=[x[0] for x in bb]
        aaa=[]
        aab=[]
        for i in bb:
            if i[0]>thr1:
                aaa.append(i[0])
                aab.append(i[1]+1)
        imgc,img3c=color_it(re,aab)
        
        img_d=cv2.dilate(imgc,cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(9,9)),iterations=3)
        aa,bb,re=count_obj2(img_d,255)
        bbb=[x[0] for x in bb]
        aaa=[]
        aab=[]
        for i in bb:
            if i[0]>thr2:
                aaa.append(i[0])
                aab.append(i[1]+1)
        imgc,img3c=color_it(re,aab)
        # plt.imshow(img3c)
        # plt.show()
        plt.imsave(j,img3c)
        plt.imsave(jj,imgc)
        print(j)
        # bbbb.append(bb)
    # return bbbb
    if ret:
        img_arr.append(img3c)
        return img_arr

In [ ]:
arry=[]
for i in range(1,count+1):
    arry.append(directorym+"/frame{}.jpg".format(i))

In [ ]:
join_disconnected(arry,thr=50,thr1=3000,thr2=5000,dil=10)
